Licensed under the Apache License, Version 2.0 (the "License"); # you may not use this file except in compliance with the License. # You may obtain a copy of the License at # # http://www.apache.org/licenses/LICENSE-2.0 # # Unless required by applicable law or agreed to in writing, software # distributed under the License is distributed on an "AS IS" BASIS, # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. # See the License for the specific language governing permissions and # limitations under the License.

Licensed under the Creative Commons Attribution-NonCommercial 4.0 International License. You may not use this file except in compliance with the License. You may obtain a copy of the License at http://creativecommons.org/licenses/by-nc/4.0/ Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This project includes code from<br>
(https://github.com/lllyasviel/controlnet), licensed under the Apache License Version 2.0.<br>
(https://github.com/facebookresearch/DiT), licensed under the Attribution-NonCommercial 4.0 International.


paper:<br>
Adding Conditional Control to Text-to-Image Diffusion Models https://arxiv.org/abs/2302.05543.<br>
Scalable Diffusion Models with Transformers https://arxiv.org/abs/2212.09748.

Optuna:<br>
@inproceedings{akiba2019optuna,
  title={{O}ptuna: A Next-Generation Hyperparameter Optimization Framework},
  author={Akiba, Takuya and Sano, Shotaro and Yanase, Toshihiko and Ohta, Takeru and Koyama, Masanori},
  booktitle={The 25th ACM SIGKDD International Conference on Knowledge Discovery \& Data Mining},
  pages={2623--2631},
  year={2019}
}<br>
License:<br>
MIT License (see LICENSE).<br>
Optuna uses the codes from SciPy and fdlibm projects (see LICENSE_THIRD_PARTY).<br>

captum:<br>
License:<br>
Captum is BSD licensed, as found in the LICENSE file.<br>

Rectified Flow:<br>
paper: https://arxiv.org/abs/2209.03003<br>
github: https://github.com/gnobitab/RectifiedFlow<br>

project description:<br>
add DiT before the control net input blocks.<br>
adjust to Rectified Flow.<br>
Github: https://github.com/Allen33669/control_net_dit

install condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
import condacolab
condacolab.check()

download the source code.
ControlNet, DiT

In [ ]:
!git clone https://github.com/lllyasviel/ControlNet.git
%cd /content/ControlNet

In [ ]:
!git clone https://github.com/facebookresearch/DiT.git

update<br>
/content/ControlNet/cldm/cldm.py<br>
/content/ControlNet/cldm/ddim_hacked.py<br>
/content/ControlNet/models/cldm_v15.yaml (This is so that you can just use an L4 to execute, you can also use original cldm_v15.yaml)<br>
/content/ControlNet/ldm/models/diffusion/ddpm.py<br>
/content/ControlNet/tool_add_control.py<br>
/content/ControlNet/my_dataset_infer.py<br>

upload<br>
/content/ControlNet/control_net_DiT_environment_2.yml<br>
/content/ControlNet/environment_2.txt



editted the pinned file /usr/local/conda-meta/pinned

In [ ]:
!conda env update -n base -f control_net_DiT_environment_2.yml

In [ ]:
!conda --version

In [ ]:
!pip install -r environment_2.txt

In [ ]:
!pip install --upgrade einops
!pip install matplotlib
!pip install optuna
!pip install captum
!pip install scikit-learn

In [ ]:
!conda list

add package path

In [ ]:
sys.path.append('/usr/local/lib/python3.8/site-packages')
print(sys.path)

In [ ]:
%cd /content/ControlNet

import library

In [ ]:
import os
from huggingface_hub import hf_hub_download
import zipfile
import json
from google.colab.patches import cv2_imshow
import config
import cv2
import einops
import numpy as np
import torch
import random
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import DeviceStatsMonitor
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
import optuna
from optuna.importance import get_param_importances
from optuna.visualization import *
"""
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline
"""
from captum._utils.models.linear_model import SkLearnLinearModel
from sklearn.metrics.pairwise import cosine_similarity
from captum.attr import *

import gradio as gr
from share import *
from annotator.util import resize_image, HWC3
from annotator.canny import CannyDetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler
from tutorial_dataset import MyDataset
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict


from my_dataset_infer import MyDatasetInfer
from cldm.cldm import XAIForwardImage


In [ ]:
!nvidia-smi
torch.cuda.is_available()

functions

In [ ]:
# This function is from https://github.com/lllyasviel/controlnet
def process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, low_threshold, high_threshold):
    with torch.no_grad():
        img = resize_image(HWC3(input_image), image_resolution)
        H, W, C = img.shape

        detected_map = apply_canny(img, low_threshold, high_threshold)
        detected_map = HWC3(detected_map)

        control = torch.from_numpy(detected_map.copy()).float().cuda() / 255.0
        control = torch.stack([control for _ in range(num_samples)], dim=0)
        control = einops.rearrange(control, 'b h w c -> b c h w').clone()

        if seed == -1:
            seed = random.randint(0, 65535)
        seed_everything(seed)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        print(control.device)

        cond = {"c_concat": [control], "c_crossattn": [model.get_learned_conditioning([prompt + ', ' + a_prompt] * num_samples)]}
        un_cond = {"c_concat": None if guess_mode else [control], "c_crossattn": [model.get_learned_conditioning([n_prompt] * num_samples)]}
        shape = (4, H // 8, W // 8)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=True)

        model.control_scales = [strength * (0.825 ** float(12 - i)) for i in range(13)] if guess_mode else ([strength] * 13)  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
        samples, intermediates = ddim_sampler.sample(ddim_steps, num_samples,
                                                     shape, cond, verbose=False, eta=eta,
                                                     unconditional_guidance_scale=scale,
                                                     unconditional_conditioning=un_cond)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        x_samples = model.decode_first_stage(samples)
        x_samples = (einops.rearrange(x_samples, 'b c h w -> b h w c') * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)

        results = [x_samples[i] for i in range(num_samples)]
    return [255 - detected_map] + results

dataset

dataset > download and unzip dataset

In [ ]:
directory_name = "training"
os.makedirs(directory_name, exist_ok=True)

In [ ]:
#download dataset
repo_id = "lllyasviel/ControlNet"
filename = "training/fill50k.zip"
file_path = hf_hub_download(repo_id=repo_id, filename=filename)
print("Downloaded file path:", file_path)

In [ ]:
#unzip the dataset
extract_dir = "/content/ControlNet/training"
with zipfile.ZipFile(file_path, 'r') as zip_ref:
   zip_ref.extractall(extract_dir)

dataset > explore dataset

In [ ]:
print_img_data = []
with open('./training/fill50k/prompt.json', 'rt') as f:
  for line in f:
    print_img_data.append(json.loads(line))

item = print_img_data[321]

source_filename = item['source']
target_filename = item['target']
prompt = item['prompt']

source = cv2.imread('./training/fill50k/' + source_filename)
target = cv2.imread('./training/fill50k/' + target_filename)

print(prompt)
cv2_imshow(source)
cv2_imshow(target)

dataset > create and set dataset

In [ ]:
dataset = MyDataset()
print(len(dataset))
dataset.data = dataset.data[:20]
print(len(dataset))

item = dataset[3]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

train

train > download ckpt

In [ ]:
repo_id = "botp/stable-diffusion-v1-5"
filename = "v1-5-pruned.ckpt"
file_path = hf_hub_download(repo_id=repo_id, filename=filename)
print("Downloaded file path:", file_path)

train > load pretrained params

In [ ]:
!python tool_add_control.py {file_path} ./models/control_sd15_ini.ckpt

train > find hyperparameters > find hyperparameters

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1)
    l1_lambda = trial.suggest_float("l1_lambda", 1e-5, 1)
    l2_lambda = trial.suggest_float("l2_lambda", 1e-5, 1)
    batch_size = trial.suggest_int("batch_size", 1, 4)

    resume_path = './models/control_sd15_ini.ckpt'
    logger_freq = 1
    sd_locked = True
    only_mid_control = False

    model = create_model('./models/cldm_v15.yaml')
    model.load_state_dict(load_state_dict(resume_path, location='cpu'), strict=False)
    model.learning_rate = learning_rate
    model.sd_locked = sd_locked
    model.only_mid_control = only_mid_control
    model.l1_lambda = l1_lambda
    model.l2_lambda = l2_lambda

    dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
    logger = ImageLogger(batch_frequency=logger_freq)
    trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger], max_epochs=1, log_every_n_steps=2)

    hyperparameters = dict(learning_rate=learning_rate)
    trainer.logger.log_hyperparams(hyperparameters)
    trainer.fit(model, dataloader)

    return trainer.callback_metrics["train/loss"].item(), trainer.callback_metrics["train/l1_regular_loss"].item(), trainer.callback_metrics["train/l2_regular_loss"].item()

In [ ]:
    study = optuna.create_study(
        directions=['minimize', 'minimize', 'minimize'],
        sampler=optuna.samplers.NSGAIISampler()
    )
    print(f"Sampler is {study.sampler.__class__.__name__}")
    study.enqueue_trial(
      {
        "learning_rate": 1e-5,
        "l1_lambda": 1e-3,
        "l2_lambda": 1e-3,
      }
    )
    study.optimize(objective, n_trials=2)

    print("Number of finished trials: {}".format(len(study.trials)))

In [ ]:
for trial in study.trials:
  print('_'* 60)
  print(f"Trial {trial.number}:")
  print(f" Params: {trial.params}")
  print(f" Values: {trial.values}")
  print(f" State: {trial.state}")
  print(f" Duration: {trial.duration}")
  print(f" User Attributes: {trial.user_attrs}")
  print(f" System Attributes: {trial.system_attrs}")

importances = get_param_importances(study, target=lambda t: t.values[0])
print('train/loss importances:')
print(importances)

importances = get_param_importances(study, target=lambda t: t.values[1])
print('train/l1_regular_loss:')
print(importances)

importances = get_param_importances(study, target=lambda t: t.values[2])
print('train/l2_regular_loss:')
print(importances)


train > find hyperparameters > visualize trial and hyperparameters

In [ ]:
fig = plot_param_importances(study)
fig.show()
fig = plot_optimization_history(study, target=lambda t: t.values[0])
fig.show()
fig = plot_optimization_history(study, target=lambda t: t.values[1])
fig.show()
fig = plot_optimization_history(study, target=lambda t: t.values[2])
fig.show()
fig = plot_intermediate_values(study)
fig.show()
fig = plot_parallel_coordinate(study, target=lambda t: t.values[0])
fig.show()
fig = plot_parallel_coordinate(study, target=lambda t: t.values[1])
fig.show()
fig = plot_parallel_coordinate(study, target=lambda t: t.values[2])
fig.show()
fig = plot_parallel_coordinate(study, params=["learning_rate", "l1_lambda", "l2_lambda"], target=lambda t: t.values[0])
fig.show()
fig = plot_parallel_coordinate(study, params=["learning_rate", "l1_lambda", "l2_lambda"], target=lambda t: t.values[1])
fig.show()
fig = plot_parallel_coordinate(study, params=["learning_rate", "l1_lambda", "l2_lambda"], target=lambda t: t.values[2])
fig.show()
fig = plot_contour(study, target=lambda t: t.values[0])
fig.show()
fig = plot_contour(study, target=lambda t: t.values[1])
fig.show()
fig = plot_contour(study, target=lambda t: t.values[2])
fig.show()
fig = plot_slice(study, target=lambda t: t.values[0])
fig.show()
fig = plot_slice(study, target=lambda t: t.values[1])
fig.show()
fig = plot_slice(study, target=lambda t: t.values[2])
fig.show()
fig = plot_edf(study, target=lambda t: t.values[0])
fig.show()
fig = plot_edf(study, target=lambda t: t.values[1])
fig.show()
fig = plot_edf(study, target=lambda t: t.values[2])
fig.show()
fig = plot_rank(study, target=lambda t: t.values[0])
fig.show()
fig = plot_rank(study, target=lambda t: t.values[1])
fig.show()
fig = plot_rank(study, target=lambda t: t.values[2])
fig.show()
fig = plot_timeline(study)
fig.show()


train > Rectified Flow > first Rectified Flow > train the model

In [ ]:
# This is adapted from https://github.com/lllyasviel/controlnet
# Configs
resume_path = './models/control_sd15_ini.ckpt'
batch_size = 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v15.yaml')
model.load_state_dict(load_state_dict(resume_path, location='cpu'), strict=False)
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger, DeviceStatsMonitor()], max_epochs=1, log_every_n_steps=2)


# Train!
trainer.fit(model, dataloader)

train > save the model ckpt file

In [ ]:
checkpoint = {'model_state_dict': model.state_dict() }
torch.save(checkpoint, './models/sd15_control_net_DiT_fill50k_rectified_flow_1.ckpt')

train > Rectified Flow > first Rectified Flow > generate samples

In [ ]:
ckpt_path = './models/sd15_control_net_DiT_fill50k_rectified_flow_1.ckpt'
ckpt = torch.load(ckpt_path)
model = create_model('./models/cldm_v15.yaml')
model.load_state_dict(ckpt['model_state_dict'])
model = model.cuda()

apply_canny = CannyDetector()
ddim_sampler = DDIMSampler(model)

In [ ]:
my_dataset = MyDatasetInfer()
texts = []
images = []
images_target = []
images_guided = []

for i in range(5):
  item = my_dataset[i]
  input_image_target = item['jpg']
  prompt = item['txt']
  input_image = item['hint']

  a_prompt = 'best quality, extremely detailed'
  n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
  num_samples = 3
  image_resolution = 512
  ddim_steps = 20
  guess_mode = False
  strength = 1
  scale = 9
  seed = 10000
  eta = 0
  low_threshold = 100
  high_threshold = 200

  result = process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, low_threshold, high_threshold)
  print('_' * 60)
  print(f'result:{len(result)}, {result[0].shape}')
  for i in range(len(result)):
    print(result[i].shape)
    plt.imshow(result[i])
    plt.axis('off') # Hide the axis
    plt.show()

  for i in range(1,len(result)):
    texts.append(prompt)
    images.append(result[i])
    images_target.append(input_image_target)
    images_guided.append(input_image)

print('_' * 60)
print(f'texts:{len(texts)}')
print(texts[0])
print(texts[1])
print(texts[3])
print(texts[10])
print(f'images_guided:{len(images_guided)}, {images_guided[0].shape}')
plt.imshow(images_guided[0])
plt.show()
plt.imshow(images_guided[1])
plt.show()
plt.imshow(images_guided[3])
plt.show()
plt.imshow(images_guided[10])
plt.show()
print(f'images:{len(images)}, {images[0].shape}')
plt.imshow(images[0])
plt.show()
plt.imshow(images[1])
plt.show()
plt.imshow(images[3])
plt.show()
plt.imshow(images[10])
plt.show()
print(f'images_target:{len(images_target)}, {images_target[0].shape}')
plt.imshow(images_target[0])
plt.show()
plt.imshow(images_target[1])
plt.show()
plt.imshow(images_target[3])
plt.show()
plt.imshow(images_target[10])
plt.show()

infer_prompt_guided_predict_target = [texts, images_guided, images, images_target]
"""
infer_prompt_guided_predict_target.pth: [texts, images_guided, images, images_target]
texts: list of strings
images_guided: list of guided images, each image is a numpy array of shape (512, 512, 3)
images: list of predicted images, each image is a numpy array of shape (512, 512, 3)
images_target: list of groud truth images, each image is a numpy array of shape (512, 512, 3)
"""
torch.save(infer_prompt_guided_predict_target, 'infer_prompt_guided_predict_target_rectified_flow_1.pth')

train > Rectified Flow > second Rectified Flow > reflow > train the model

In [ ]:
from torch.utils.data import Dataset

class MyDatasetRectifiedFlow(Dataset):
    def __init__(self, data_path):
        self.data = []
        data = torch.load(data_path)
        texts = data[0]
        images_guided = data[1]
        images_predict = data[2]
        images_target = data[3]
          for i in range(len(texts)):
              item = {}
              item['txt'] = texts[i]
              item['hint'] = images_guided[i]
              item['jpg'] = images_predict[i]
              item['target'] = images_target[i]
              self.data.append(item)

        images_guided_tensor = torch.tensor(images_guided)
        images_predict_tensor = torch.tensor(images_predict)
        images_target_tensor = torch.tensor(images_target)

        print('_' * 60)
        print(f'texts:{len(texts)}')
        print(texts[0])
        print(texts[1])
        print(texts[3])
        print(texts[10])
        print(f'images_guided:{len(images_guided)}, {images_guided[0].shape}')
        print(f'images_guided:{images_guided[0]}')
        plt.imshow(images_guided[0])
        plt.show()
        plt.imshow(images_guided[1])
        plt.show()
        plt.imshow(images_guided[3])
        plt.show()
        plt.imshow(images_guided[10])
        plt.show()
        print(f'images_predict:{len(images_predict)}, {images_predict[0].shape}')
        print(f'images_guided:{images_predict[0]}')
        plt.imshow(images_predict[0])
        plt.show()
        plt.imshow(images_predict[1])
        plt.show()
        plt.imshow(images_predict[3])
        plt.show()
        plt.imshow(images_predict[10])
        plt.show()
        print(f'images_target:{len(images_target)}, {images_target[0].shape}')
        print(f'images_guided:{images_target[0]}')
        plt.imshow(images_target[0])
        plt.show()
        plt.imshow(images_target[1])
        plt.show()
        plt.imshow(images_target[3])
        plt.show()
        plt.imshow(images_target[10])
        plt.show()

        print('_' * 60)
        print(f'images_guided_tensor:{images_guided_tensor.shape}')
        print(f'images_predict_tensor:{images_predict_tensor.shape}')
        print(f'images_target_tensor:{images_target_tensor.shape}')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source = item['hint']
        target = item['jpg']
        prompt = item['txt']

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)

In [ ]:
dataset = MyDatasetRectifiedFlow('infer_prompt_guided_predict_target_rectified_flow_1.pth')
print(len(dataset))
dataset.data = dataset.data[:20]
print(len(dataset))

item = dataset[3]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

In [ ]:
# This is adapted from https://github.com/lllyasviel/controlnet
# Configs
resume_path = './models/sd15_control_net_DiT_fill50k_rectified_flow_1.ckpt'
batch_size = 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v15.yaml')
model.load_state_dict(load_state_dict(resume_path, location='cpu'), strict=False)
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger, DeviceStatsMonitor()], max_epochs=1, log_every_n_steps=2)


# Train!
trainer.fit(model, dataloader)

In [ ]:
checkpoint = {'model_state_dict': model.state_dict() }
torch.save(checkpoint, './models/sd15_control_net_DiT_fill50k_rectified_flow_2.ckpt')

train > Rectified Flow > second Rectified Flow > generate samples

In [ ]:
ckpt_path = './models/sd15_control_net_DiT_fill50k_rectified_flow_2.ckpt'
ckpt = torch.load(ckpt_path)
model = create_model('./models/cldm_v15.yaml')
model.load_state_dict(ckpt['model_state_dict'])
model = model.cuda()

apply_canny = CannyDetector()
ddim_sampler = DDIMSampler(model)

In [ ]:
my_dataset = MyDatasetInfer()
texts = []
images = []
images_target = []
images_guided = []

for i in range(5):
  item = my_dataset[i]
  input_image_target = item['jpg']
  prompt = item['txt']
  input_image = item['hint']

  a_prompt = 'best quality, extremely detailed'
  n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
  num_samples = 3
  image_resolution = 512
  ddim_steps = 20
  guess_mode = False
  strength = 1
  scale = 9
  seed = 10000
  eta = 0
  low_threshold = 100
  high_threshold = 200

  result = process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, low_threshold, high_threshold)
  print('_' * 60)
  print(f'result:{len(result)}, {result[0].shape}')
  for i in range(len(result)):
    print(result[i].shape)
    plt.imshow(result[i])
    plt.axis('off') # Hide the axis
    plt.show()

  for i in range(1,len(result)):
    texts.append(prompt)
    images.append(result[i])
    images_target.append(input_image_target)
    images_guided.append(input_image)

print('_' * 60)
print(f'texts:{len(texts)}')
print(texts[0])
print(texts[1])
print(texts[3])
print(texts[10])
print(f'images_guided:{len(images_guided)}, {images_guided[0].shape}')
plt.imshow(images_guided[0])
plt.show()
plt.imshow(images_guided[1])
plt.show()
plt.imshow(images_guided[3])
plt.show()
plt.imshow(images_guided[10])
plt.show()
print(f'images:{len(images)}, {images[0].shape}')
plt.imshow(images[0])
plt.show()
plt.imshow(images[1])
plt.show()
plt.imshow(images[3])
plt.show()
plt.imshow(images[10])
plt.show()
print(f'images_target:{len(images_target)}, {images_target[0].shape}')
plt.imshow(images_target[0])
plt.show()
plt.imshow(images_target[1])
plt.show()
plt.imshow(images_target[3])
plt.show()
plt.imshow(images_target[10])
plt.show()

infer_prompt_guided_predict_target = [texts, images_guided, images, images_target]
"""
infer_prompt_guided_predict_target.pth: [texts, images_guided, images, images_target]
texts: list of strings
images_guided: list of guided images, each image is a numpy array of shape (512, 512, 3)
images: list of predicted images, each image is a numpy array of shape (512, 512, 3)
images_target: list of groud truth images, each image is a numpy array of shape (512, 512, 3)
"""
torch.save(infer_prompt_guided_predict_target, 'infer_prompt_guided_predict_target_rectified_flow_2.pth')

train > Rectified Flow > distill<br>
use the original model (can adjust euler step to 1 for distillation, if you want)

In [ ]:
dataset = MyDatasetRectifiedFlow('infer_prompt_guided_predict_target_rectified_flow_2.pth')
print(len(dataset))
dataset.data = dataset.data[:20]
print(len(dataset))

item = dataset[3]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

In [ ]:
# This is adapted from https://github.com/lllyasviel/controlnet
# Configs
resume_path = './models/sd15_control_net_DiT_fill50k_rectified_flow_2.ckpt'
batch_size = 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v15.yaml')
model.load_state_dict(load_state_dict(resume_path, location='cpu'), strict=False)
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger, DeviceStatsMonitor()], max_epochs=1, log_every_n_steps=2)


# Train!
trainer.fit(model, dataloader)

In [ ]:
checkpoint = {'model_state_dict': model.state_dict() }
torch.save(checkpoint, './models/sd15_control_net_DiT_fill50k.ckpt')

XAI > prepare sample and model

In [ ]:
dataset = MyDataset()
print(len(dataset))
dataset.data = dataset.data[:20]
print(len(dataset))

item = dataset[3]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

batch_size = 1
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
data_iter = iter(dataloader)
sample_batch = next(data_iter)

In [ ]:
ckpt_path = './models/sd15_control_net_DiT_fill50k.ckpt'
ckpt = torch.load(ckpt_path)
model = create_model('./models/cldm_v15.yaml')
model.load_state_dict(ckpt['model_state_dict'])
model = model.cuda()
model.lime_sample = sample_batch

logger_freq = 300
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger, DeviceStatsMonitor()], max_epochs=1, log_every_n_steps=2)

XAI > Kernel SHAP > input text

In [ ]:
input_text = model.lime_sample['txt']
input_text_words = input_text[0].split()
input_text_words_tensor = torch.ones(len(input_text_words)).unsqueeze(0)

ks = KernelShap(model.lime_text_0_1)
attr_coefs = ks.attribute(input_text_words_tensor, n_samples=10)
print(attr_coefs)

XAI > Kernel SHAP > input image square

In [ ]:
input_hint = model.lime_sample['hint']
print(f'input_hint.shape:{input_hint.shape}')

#feature mask
mask_size = 16
feature_mask = torch.zeros_like(input_hint[0], dtype=torch.long)

# Fill the feature mask with square regions
for i in range(0, feature_mask.shape[0], mask_size):
  for j in range(0, feature_mask.shape[1], mask_size):
     for k in range(0, 3):
       feature_mask[i:i+mask_size, j:j+mask_size, k] = (i // mask_size) * (feature_mask.shape[1] // mask_size) + (j // mask_size)

feature_mask = feature_mask.unsqueeze_(0)

#attribute
ks = KernelShap(model.kernel_shap_image)
attr_coefs = ks.attribute(input_hint, n_samples=10, feature_mask=feature_mask)
print(attr_coefs)

In [ ]:
attr_coefs_graph = attr_coefs.squeeze(0)
attr_coefs_graph = attr_coefs_graph.permute(2, 0, 1)

split_tensors = torch.split(attr_coefs_graph, 1, dim=0)
for i, chunk in enumerate(split_tensors):
  chunk = chunk.squeeze(0)
  chunk = chunk.detach().numpy()

  plt.figure(figsize=(10, 5))
  plt.imshow(chunk, cmap='hot', interpolation='nearest')
  plt.colorbar()
  plt.title("Attributions")
  plt.show()

XAI > LIME > input text

In [ ]:
def similarity_kernel(
     original_input,
     perturbed_input,
     perturbed_interpretable_input,
     **kwargs):
         # kernel_width will be provided to attribute as a kwarg
         kernel_width = kwargs["kernel_width"]
         result = cosine_similarity(original_input, perturbed_input).squeeze(0)[0]
         result = (result + 1) / 2
         return result

def perturb_func(
     original_input,
     **kwargs):
     input = torch.randint(0, 2, original_input.squeeze(0).shape)
     input = input.unsqueeze(0)
     return input

def to_interp_transform(curr_sample, original_inp, **kwargs):
     return curr_sample

In [ ]:
input_text = model.lime_sample['txt']
input_text_words = input_text[0].split()
input_text_words_tensor = torch.ones(len(input_text_words)).unsqueeze(0)

lime_attr = LimeBase(model.lime_text_0_1,
                         SkLearnLinearModel("linear_model.Ridge"),
                         similarity_func=similarity_kernel,
                         perturb_func=perturb_func,
                         perturb_interpretable_space=False,
                         from_interp_rep_transform=None,
                         to_interp_rep_transform=to_interp_transform)

attr_coefs = lime_attr.attribute(input_text_words_tensor, kernel_width=1.1, n_samples=10)
print(f'attr_coefs.shape:{attr_coefs.shape}')
print(f'attr_coefs:{attr_coefs}')

XAI > LIME > input image

In [ ]:
def similarity_kernel(
     original_input,
     perturbed_input,
     perturbed_interpretable_input,
     **kwargs):
         # kernel_width will be provided to attribute as a kwarg
         kernel_width = kwargs["kernel_width"]
         result = cosine_similarity(original_input, perturbed_input).squeeze(0)[0]
         result = (result + 1) / 2
         return result

def perturb_func(
     original_input,
     **kwargs):
     input = original_input.squeeze()
     noise = torch.floor(torch.randn_like(input) * 64)
     input = noise + 128
     input = torch.clamp(input, min=0, max=255)
     input = input.unsqueeze(0)
     return input

def to_interp_transform(curr_sample, original_inp, **kwargs):
     return curr_sample

In [ ]:
input_hint = model.lime_sample['hint']
input_hint = input_hint.squeeze(0)
input_hint = torch.flatten(input_hint)
input_hint = input_hint.unsqueeze(0)

lime_attr = LimeBase(model.lime_image,
                         SkLearnLinearModel("linear_model.Ridge"),
                         similarity_func=similarity_kernel,
                         perturb_func=perturb_func,
                         perturb_interpretable_space=False,
                         from_interp_rep_transform=None,
                         to_interp_rep_transform=to_interp_transform)

attr_coefs = lime_attr.attribute(input_hint, kernel_width=1.1, n_samples=10)

In [ ]:
attr_coefs = attr_coefs.view(model.lime_sample['hint'].shape).squeeze().detach().numpy()

plt.figure(figsize=(10, 5))
plt.imshow(attr_coefs, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title("Attributions")
plt.show()

XAI > Feature Ablation > input image

In [ ]:
input_hint = model.lime_sample['hint']
print(f'input_hint.shape:{input_hint.shape}')

#feature mask
mask_size = 16
feature_mask = torch.zeros_like(input_hint[0], dtype=torch.long)

# Fill the feature mask with square regions
for i in range(0, feature_mask.shape[0], mask_size):
  for j in range(0, feature_mask.shape[1], mask_size):
     for k in range(0, 3):
       feature_mask[i:i+mask_size, j:j+mask_size, k] = (i // mask_size) * (feature_mask.shape[1] // mask_size) + (j // mask_size)

feature_mask = feature_mask.unsqueeze_(0)

feature_perm = FeatureAblation(model.kernel_shap_image)
attr_coefs = feature_perm.attribute(input_hint, feature_mask=feature_mask)
print(f'attr_coefs.shape: {attr_coefs.shape}')
print(f'attr_coefs: {attr_coefs}')


In [ ]:
attr_coefs_graph = attr_coefs.squeeze(0)
attr_coefs_graph = attr_coefs_graph.permute(2, 0, 1)

split_tensors = torch.split(attr_coefs_graph, 1, dim=0)
for i, chunk in enumerate(split_tensors):
  chunk = chunk.squeeze(0)
  chunk = chunk.detach().numpy()

  plt.figure(figsize=(10, 5))
  plt.imshow(chunk, cmap='hot', interpolation='nearest')
  plt.colorbar()
  plt.title("Attributions")
  plt.show()

XAI > Layer Feature Ablation > input image

In [ ]:
input_hint = model.lime_sample['hint']
print(f'input_hint.shape:{input_hint.shape}')

#feature mask
mask_size = 2
feature_mask = torch.zeros((4, 64, 64), dtype=torch.long)

# Fill the feature mask with square regions
for k in range(0, feature_mask.shape[0], 1):
  for i in range(0, feature_mask.shape[1], mask_size):
    for j in range(0, feature_mask.shape[2], mask_size):
      feature_mask[k, i:i+mask_size, j:j+mask_size] = (i // mask_size) * (feature_mask.shape[2] // mask_size) + (j // mask_size)

feature_mask = feature_mask.to('cuda')

ablator = LayerFeatureAblation(model.kernel_shap_image, model.model.diffusion_model.out[2])
attr_coefs = ablator.attribute(input_hint, layer_mask=feature_mask)
print(f'attr_coefs.shape: {attr_coefs.shape}')
print(f'attr_coefs: {attr_coefs}')


In [ ]:
attr_coefs_graph = attr_coefs.squeeze(0)

split_tensors = torch.split(attr_coefs_graph, 1, dim=0)
for i, chunk in enumerate(split_tensors):
  chunk = chunk.squeeze(0)
  chunk = chunk.detach().cpu().numpy()

  plt.figure(figsize=(10, 5))
  plt.imshow(chunk, cmap='hot', interpolation='nearest')
  plt.colorbar()
  plt.title("Attributions")
  plt.show()


XAI > Neuron Feature Ablation > input image

In [ ]:
input_hint = model.lime_sample['hint']
print(f'input_hint.shape:{input_hint.shape}')

#feature mask
mask_size = 16
feature_mask = torch.zeros_like(input_hint[0], dtype=torch.long)

# Fill the feature mask with square regions
for i in range(0, feature_mask.shape[0], mask_size):
  for j in range(0, feature_mask.shape[1], mask_size):
     for k in range(0, 3):
       feature_mask[i:i+mask_size, j:j+mask_size, k] = (i // mask_size) * (feature_mask.shape[1] // mask_size) + (j // mask_size)

feature_mask = feature_mask.unsqueeze_(0)

ablator = NeuronFeatureAblation(model.kernel_shap_image, model.model.diffusion_model.out[2])
attr_coefs = ablator.attribute(input_hint, neuron_selector=(3,1,2), feature_mask=feature_mask)
print(f'attr_coefs.shape: {attr_coefs.shape}')
print(f'attr_coefs: {attr_coefs}')

In [ ]:
attr_coefs_graph = attr_coefs.squeeze(0)
attr_coefs_graph = attr_coefs_graph.permute(2, 0, 1)

split_tensors = torch.split(attr_coefs_graph, 1, dim=0)
for i, chunk in enumerate(split_tensors):
  chunk = chunk.squeeze(0)
  chunk = chunk.detach().numpy()

  plt.figure(figsize=(10, 5))
  plt.imshow(chunk, cmap='hot', interpolation='nearest')
  plt.colorbar()
  plt.title("Attributions")
  plt.show()

XAI > Feature Permutation > input image

In [ ]:
feature_permutation_batch_size = 2
feature_permutation_dataloader = DataLoader(dataset, num_workers=0, batch_size=feature_permutation_batch_size, shuffle=True)
feature_permutation_data_iter = iter(feature_permutation_dataloader)
feature_permutation_sample_batch = next(feature_permutation_data_iter)

model.feature_permutation_sample = feature_permutation_sample_batch

In [ ]:
input_hint = model.feature_permutation_sample['hint']
print(f'input_hint.shape:{input_hint.shape}')

#feature mask
mask_size = 16
feature_mask = torch.zeros_like(input_hint[0], dtype=torch.long)

# Fill the feature mask with square regions
for i in range(0, feature_mask.shape[0], mask_size):
  for j in range(0, feature_mask.shape[1], mask_size):
     for k in range(0, 3):
       feature_mask[i:i+mask_size, j:j+mask_size, k] = (i // mask_size) * (feature_mask.shape[1] // mask_size) + (j // mask_size)

feature_mask = feature_mask.unsqueeze_(0)

feature_perm = FeaturePermutation(model.kernel_shap_image)
attr_coefs = feature_perm.attribute(input_hint, feature_mask=feature_mask)
print(f'attr_coefs.shape: {attr_coefs.shape}')
print(f'attr_coefs: {attr_coefs}')

In [ ]:
attr_coefs_graph = attr_coefs.squeeze(0)
attr_coefs_graph = attr_coefs_graph.permute(2, 0, 1)

split_tensors = torch.split(attr_coefs_graph, 1, dim=0)
for i, chunk in enumerate(split_tensors):
  chunk = chunk.squeeze(0)
  chunk = chunk.detach().numpy()

  plt.figure(figsize=(10, 5))
  plt.imshow(chunk, cmap='hot', interpolation='nearest')
  plt.colorbar()
  plt.title("Attributions")
  plt.show()

infer

In [ ]:
ckpt_path = './models/sd15_control_net_DiT_fill50k.ckpt'
ckpt = torch.load(ckpt_path)
model = create_model('./models/cldm_v15.yaml')
model.load_state_dict(ckpt['model_state_dict'])
model = model.cuda()

apply_canny = CannyDetector()
ddim_sampler = DDIMSampler(model)

In [ ]:
my_dataset = MyDatasetInfer()
item = my_dataset[123]
input_image_target = item['jpg']
prompt = item['txt']
input_image = item['hint']
print(prompt)
print(input_image_target.shape)
print(input_image.shape)

a_prompt = 'best quality, extremely detailed'
n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
num_samples = 3
image_resolution = 512
ddim_steps = 20
guess_mode = False
strength = 1
scale = 9
seed = 10000
eta = 0
low_threshold = 100
high_threshold = 200

result = process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, low_threshold, high_threshold)
print('result:')
print(result)
for i in range(len(result)):
  print(result[i].shape)
  plt.imshow(result[i])
  plt.axis('off') # Hide the axis
  plt.show()


prepare evaluation dataset

In [ ]:
my_dataset = MyDatasetInfer()
texts = []
images = []
images_target = []
images_guided = []

for i in range(5):
  item = my_dataset[i]
  input_image_target = item['jpg']
  prompt = item['txt']
  input_image = item['hint']

  a_prompt = 'best quality, extremely detailed'
  n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
  num_samples = 3
  image_resolution = 512
  ddim_steps = 20
  guess_mode = False
  strength = 1
  scale = 9
  seed = 10000
  eta = 0
  low_threshold = 100
  high_threshold = 200

  result = process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, low_threshold, high_threshold)
  print('_' * 60)
  print(f'result:{len(result)}, {result[0].shape}')
  for i in range(len(result)):
    print(result[i].shape)
    plt.imshow(result[i])
    plt.axis('off') # Hide the axis
    plt.show()

  for i in range(1,len(result)):
    texts.append(prompt)
    images.append(result[i])
    images_target.append(input_image_target)
    images_guided.append(input_image)

print('_' * 60)
print(f'texts:{len(texts)}')
print(texts[0])
print(texts[1])
print(texts[3])
print(texts[10])
print(f'images_guided:{len(images_guided)}, {images_guided[0].shape}')
plt.imshow(images_guided[0])
plt.show()
plt.imshow(images_guided[1])
plt.show()
plt.imshow(images_guided[3])
plt.show()
plt.imshow(images_guided[10])
plt.show()
print(f'images:{len(images)}, {images[0].shape}')
plt.imshow(images[0])
plt.show()
plt.imshow(images[1])
plt.show()
plt.imshow(images[3])
plt.show()
plt.imshow(images[10])
plt.show()
print(f'images_target:{len(images_target)}, {images_target[0].shape}')
plt.imshow(images_target[0])
plt.show()
plt.imshow(images_target[1])
plt.show()
plt.imshow(images_target[3])
plt.show()
plt.imshow(images_target[10])
plt.show()

infer_prompt_guided_predict_target = [texts, images_guided, images, images_target]
"""
infer_prompt_guided_predict_target.pth: [texts, images_guided, images, images_target]
texts: list of strings
images_guided: list of guided images, each image is a numpy array of shape (512, 512, 3)
images: list of predicted images, each image is a numpy array of shape (512, 512, 3)
images_target: list of groud truth images, each image is a numpy array of shape (512, 512, 3)
"""
torch.save(infer_prompt_guided_predict_target, 'infer_prompt_guided_predict_target.pth')


logging > tensorboard

In [ ]:
!pip uninstall tb-nightly tensorboard tensorflow tensorflow-estimator
!pip install tensorflow

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=/content/ControlNet/lightning_logs